In [1]:
import sys
import os
import copy
current_dir = os.getcwd()
sys.path.append(current_dir[:-9])


<!-- <style>
  table {
    table-layout: fixed;
    width: 100%;
  }
  th:nth-child(6), td:nth-child(6) {
    width: 300px;          /* columns 1–5 auto, column 6 fixed */
  }
  td, th {
    overflow-wrap: break-word;
  }
</style> -->

In [2]:
import importlib

# 1) Tell the helper that we really want console logs
os.environ["LOG_TO_CONSOLE"] = "false"

In [3]:
from qiskit import QuantumCircuit, transpile

# simple test: one CX followed by a SWAP
qc = QuantumCircuit(3)
qc.cx(0,1)
qc.swap(0,2)

print("input depth:", qc.depth())

# Qiskit 1.2.4, level 0  ➜ physical SWAP kept
out12 = transpile(qc, basis_gates=['cx','rz','sx'], coupling_map=[[0,1],[1,2]], optimization_level=0)
print("1.2.4 level-0 depth:", out12.depth())   # > input depth

# Qiskit 2.0.0, level 0  ➜ SWAP absorbed, depth equal to CX decomposition
out20 = transpile(qc, basis_gates=['cx','rz','sx'], coupling_map=[[0,1],[1,2]], optimization_level=2)
print("2.0.0 level-0 depth:", out20.depth())   # smaller, SWAP gone


input depth: 2
1.2.4 level-0 depth: 25
2.0.0 level-0 depth: 7


## 5 Where (and how) `qiskit.transpile` is called in our pipeline  
  
There are **two distinct invocations** of Qiskit’s transpiler:

| call-site | purpose | `coupling_map` | optimisation level etc. |
|-----------|---------|----------------|-------------------------|
| **(A)** `QiskitCompiler.__init__ → _calibrate_swap` | learn depth of a 2-qubit SWAP in the *native* basis | **None** (no device constraints yet) | taken from the compiler constructor |
| **(B)** `QuantumDevice.allocate → compiler.transpile` | convert the *logical* circuit to native basis **and** to the device’s topology | **device.connectivity** | same compiler instance ⇒ same optimisation level / passes |

Because (A) runs **without** a coupling-map, its depth is a **lower
bound**; if later (B) finds the same SWAP takes more depth on a real
device, `_calibrate_swap` is run again and `swap_duration` is updated.

In [12]:
import networkx as nx
from qiskit.transpiler import CouplingMap
from qiskit import QuantumCircuit
from pathlib import Path

# ---- helpers from your repo (assume these are import-able) --------------
from qrew.simulation.refactor.resources.quantum_resources import QuantumDevice
from qrew.simulation.refactor.q_interop.transpilers import QiskitCompiler
from qrew.simulation.refactor.quantum import QuantumCircuit as MyCircuit
from qrew.simulation.refactor.quantum import QuantumInstruction, CX, SWAP
# -------------------------------------------------------------------------

# simple 3-line topology  0—1—2
def line_graph(n=3):
    g = nx.Graph()
    g.add_nodes_from(range(n))
    g.add_edges_from([(i, i+1) for i in range(n-1)])
    return g

# we'll build two compilers: fast (opt-0) vs heavy (opt-2)
fast_compiler  = QiskitCompiler(basis_gates=("ECR","RZ","SX","X"),
                                optimization_level=0)
light_compiler = QiskitCompiler(basis_gates=("ECR","I","RZ","SX","X","CX"),
                                optimization_level=1)
heavy_compiler = QiskitCompiler(basis_gates=("ECR","RZ","SX","X"),
                                optimization_level=2)
best_compiler = QiskitCompiler(basis_gates=("ECR","RZ","SX","X"),
                                optimization_level=3)

fast_dev  = QuantumDevice("Line-fast",  line_graph(), compiler=fast_compiler)
light_dev = QuantumDevice("Line-light", line_graph(), compiler=light_compiler)
heavy_dev = QuantumDevice("Line-heavy", line_graph(), compiler=heavy_compiler)
best_dev = QuantumDevice("Line-best", line_graph(), compiler=best_compiler)
print("Fast SWAP depth  :", fast_dev.swap_duration)
print("Light SWAP depth  :", light_dev.swap_duration)
print("Heavy SWAP depth :", heavy_dev.swap_duration)


Fast SWAP depth  : 6
Light SWAP depth  : 3
Heavy SWAP depth : 0


* The constructor of each `QiskitCompiler` ran a **calibration
  transpile** of a 2-qubit SWAP.  
  *It used `optimization_level` 0 / 2 but **no** coupling map.*
* On a basis of `{ECR,RZ,SX,X}` the lower-bound depths are  
  typically 1 for opt-0 vs 3+ for opt-2 (depends on the basis).

### Allocate a logical circuit on both devices

In [11]:
# our logical circuit:  CX – SWAP – CX  on logical qubits 0,1,2
log_circ = MyCircuit(qubit_count=3)
log_circ.add_instruction(QuantumInstruction(CX(),   (0,1)))
log_circ.add_instruction(QuantumInstruction(SWAP(), (0,2)))
log_circ.add_instruction(QuantumInstruction(CX(),   (0,1)))

from qrew.simulation.refactor.resources.quantum_resources import QuantumResource
job = QuantumResource(log_circ)
print(f"job LS config: {job.LS_parameters}")
alloc_fast  = fast_dev.allocate(job)
alloc_heavy = heavy_dev.allocate(job)
alloc_best= best_dev.allocate(job)

print("\n— Fast compiler result —")
print(alloc_fast.transpiled_circuit)
print("\n— Heavy compiler result —")
print(alloc_heavy.transpiled_circuit)
print("\n— Best compiler result —")
print(alloc_best.transpiled_circuit)

job LS config: {'transition based': True, 'epsilon': 0.3, 'objective': 'depth', 'hard_island': False}
Attempting maximal depth 1...
Attempting maximal depth 1...
Attempting maximal depth 1...

— Fast compiler result —
<QuantumCircuit 3 qubits, 70 instructions (ECR:8, RZ:40, SX:20, X:2), depth=40>

— Heavy compiler result —
<QuantumCircuit 3 qubits, 20 instructions (ECR:2, RZ:11, SX:6, X:1), depth=13>

— Best compiler result —
<QuantumCircuit 3 qubits, 20 instructions (ECR:2, RZ:11, SX:6, X:1), depth=13>


In [1]:
# ------------- TEST UTILITIES ------------------------------------------------
def run_ls_case(*, opt_level, transition_based, hard_island):
    dev = make_line_device(opt_level=opt_level)
    res, init_map, fin_map, depth, extra = dev.layout_synthesis(
        logical,
        transition_based=transition_based,
        hard_island=hard_island,
        objective="depth",
    )
    return dict(
        opt_level       = opt_level,
        transition_based= transition_based,
        hard_island     = hard_island,
        swap_duration   = dev.swap_duration,
        depth           = depth,
        swaps           = extra["swaps"],
        init_map        = init_map,
        fin_map         = fin_map,
    )

def pretty(line: dict):
    tb = "TB" if line["transition_based"] else "DP"   # transition-based / depth-prop
    hi = "HI" if line["hard_island"]      else "--"
    swaps = line["swaps"] or "∅"
    return (f"O{line['opt_level']}  {tb},{hi} │ "
            f"π₀{line['init_map']} → π_f{line['fin_map']} │ "
            f"σ={swaps} │ depth={line['depth']} (σ-cost={line['swap_duration']})")

# ------------- RUN  4×3  = 12 cases ------------------------------------------
cases = []
for lvl in (0,1,2):
    for tb in (False, True):
        for hi in (False, True):
            cases.append(run_ls_case(opt_level=lvl,
                                     transition_based=tb,
                                     hard_island=hi))

print("π/σ search results:")
for line in cases:
    print(pretty(line))

NameError: name 'make_line_device' is not defined

| device | compiler opt-level | depth after `allocate` | SWAPs inserted | notes |
|--------|-------------------|------------------------|----------------|-------|
| Line-fast  | 0 | (likely ~26) | 1 physical swap (encoded as σ≠0) | router uses `swap_duration` learned at opt-0 calibration |
| Line-heavy | 2 | (smaller, ~13) | **0** | higher-level passes merged gates and the solver found a “free” permutation |

* Because **both** calibration and allocation used `optimization_level=0`
  in the “fast” compiler, the router assumed a SWAP cost of 1 and happily
  emitted one σ-edge.  
* In the “heavy” compiler the larger calibrated `swap_duration`
  discouraged physical swaps, so the solver found a permutation that
  needs none.

## Take away
* **If you want different pass-sets for calibration vs real circuits, you
  need two compiler instances** (or expose those options separately).  
  For example, keep calibration at opt-0 to learn a *lower* bound, while
  letting user circuits compile at opt-2.
* `LayoutSynthesizer` already adapts:  
  * `swap_duration` is read from the `QuantumDevice` → affects
    `constraint_no_SWAP_before_swap_duration`.  
  * A larger `swap_duration` makes σ-solutions more expensive, so the SMT
    solver prefers *virtual* permutations (free case).  
  * If optimisation level is low and the calibrated SWAP is cheap,
    σ-solutions become acceptable, reproducing the “frozen” cost.